In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
mk=1

# Initial 

In [ ]:
import scipy.io
import numpy as np
file_name = '/content/drive/MyDrive/2022_SUMMER/Research_Burger/CODE/TranslatePythonCode/'
mat = scipy.io.loadmat(file_name + 'canadafastfood_resorted.mat')
data = mat['data']#[np.where(mtype_int==1)]
TimePeriod = len(np.unique(data[:,1]))
SamplePath = int(len(data)/TimePeriod)
print(TimePeriod,SamplePath)
print(data[:,0].reshape([TimePeriod,SamplePath]).T)

#% Labeling
MT = len(data)          #% Size of the dataset (M x T = 400 x 35 = 14000).#clusterid = np.int_(data[:,0])      % Unique market ID.
year = np.int_(data[:,1]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_aw  = np.int_(data[:,2]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_bk  = np.int_(data[:,3]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_hvy = np.int_(data[:,4]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_mcd = np.int_(data[:,5]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
N_wdy = np.int_(data[:,6]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_aw  = np.int_(data[:,7]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_bk  = np.int_(data[:,8]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_hvy = np.int_(data[:,9]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_mcd = np.int_(data[:,10]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
lagN_wdy = np.int_(data[:,11]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_aw  = np.int_(data[:,12]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_bk  = np.int_(data[:,13]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_hvy = np.int_(data[:,14]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_mcd = np.int_(data[:,15]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
fwdN_wdy = np.int_(data[:,16]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_aw  = np.int_(data[:,17]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_bk  = np.int_(data[:,18]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_hvy = np.int_(data[:,19]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_mcd = np.int_(data[:,20]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
a_wdy = np.int_(data[:,21]).reshape([TimePeriod,SamplePath]).T.reshape(-1)
pop = data[:,23].reshape([TimePeriod,SamplePath]).T.reshape(-1)
val = data[:,24].reshape([TimePeriod,SamplePath]).T.reshape(-1)
inc = data[:,25].reshape([TimePeriod,SamplePath]).T.reshape(-1)
fpop = data[:,27].reshape([TimePeriod,SamplePath]).T.reshape(-1)
finc = data[:,28].reshape([TimePeriod,SamplePath]).T.reshape(-1)
fval = data[:,29].reshape([TimePeriod,SamplePath]).T.reshape(-1)
mktfe = data[:,30].reshape([TimePeriod,SamplePath]).T.reshape(-1) #% Market fixed effect estimates (from 130913_alaToivanenWaterson3_Fixedeffect3quantile.csv/dta)
tertile = np.int_(data[:,31]).reshape([TimePeriod,SamplePath]).T.reshape(-1)       #% Market type initial guess (from 130913_alaToivanenWaterson3_Fixedeffect3quantile.csv/dta)

35 400
[[  1.   1.   1. ...   1.   1.   1.]
 [  2.   2.   2. ...   2.   2.   2.]
 [  3.   3.   3. ...   3.   3.   3.]
 ...
 [398. 398. 398. ... 398. 398. 398.]
 [399. 399. 399. ... 399. 399. 399.]
 [400. 400. 400. ... 400. 400. 400.]]


In [ ]:
def quantile(x,p):
  x = np.sort(x)
  n = len(x)
  x = np.hstack([x[0], x,x[-1]])
  i = p*n+1.5
  iu = int(np.ceil(i))
  il = int(np.floor(i))
  d = i-il
  qq = x[il-1]*(1-d)+x[iu-1]*d
  return qq

#% Number of own shops (in state space; capped at 3), from the perspective of each firm
Ni_aw = (N_aw <= 3)*N_aw + (N_aw > 3)*3
Ni_bk = (N_bk <= 3)*N_bk + (N_bk > 3)*3
Ni_hvy = (N_hvy <= 3)*N_hvy + (N_hvy > 3)*3
Ni_mcd = (N_mcd <= 3)*N_mcd + (N_mcd > 3)*3
Ni_wdy = (N_wdy <= 3)*N_wdy + (N_wdy > 3)*3

#% Number of rival shops (in state space; capped at 3), from the perspective of each firm
Nj_aw = N_bk + N_hvy + N_mcd + N_wdy
Nj_bk = N_aw + N_hvy + N_mcd + N_wdy
Nj_hvy = N_aw + N_bk + N_mcd + N_wdy
Nj_mcd = N_aw + N_bk + N_hvy + N_wdy
Nj_wdy = N_aw + N_bk + N_hvy + N_mcd
Nj_aw = (Nj_aw <= 3)*Nj_aw + (Nj_aw > 3)*3     
Nj_bk = (Nj_bk <= 3)*Nj_bk + (Nj_bk > 3)*3
Nj_hvy = (Nj_hvy <= 3)*Nj_hvy + (Nj_hvy > 3)*3
Nj_mcd = (Nj_mcd <= 3)*Nj_mcd + (Nj_mcd > 3)*3
Nj_wdy = (Nj_wdy <= 3)*Nj_wdy + (Nj_wdy > 3)*3

#% Number of pop,val,inc (in state space; capped at 3)
pop25 = quantile(pop,0.25)
val25 = quantile(val,0.25)   
inc25 = quantile(inc,0.25)  
pop50 = quantile(pop,0.5) 
val50 = quantile(val,0.5)   
inc50 = quantile(inc,0.5)  
pop75 = quantile(pop,0.75) 
val75 = quantile(val,0.75)   
inc75 = quantile(inc,0.75)   
disc_pop = np.int_(pop > pop25) + np.int_(pop > pop50) + np.int_(pop > pop75)  
disc_val = np.int_(val > val25) + np.int_(val > val50) + np.int_(val > val75)  
disc_inc = np.int_(inc > inc25) + np.int_( inc > inc50) + np.int_(inc > inc75)

#% Define state variables for each firm
RHS_aw = np.vstack([Ni_aw, Nj_aw, disc_pop,disc_val, disc_inc]).T#;         % State for A & W.      
RHS_bk = np.vstack([Ni_bk, Nj_bk, disc_pop,disc_val, disc_inc]).T#;         % State for Burger King.      
RHS_hvy = np.vstack([Ni_hvy, Nj_hvy, disc_pop,disc_val, disc_inc]).T#;      % State for Harvey's.      
RHS_mcd = np.vstack([Ni_mcd, Nj_mcd, disc_pop, disc_val,disc_inc]).T#;      % State for McDonald's.      
RHS_wdy = np.vstack( [Ni_wdy, Nj_wdy, disc_pop,disc_val, disc_inc]).T#;

In [ ]:
stateID_aw = Ni_aw*4**4+Nj_aw*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_bk = Ni_bk*4**4+Nj_bk*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_hvy = Ni_hvy*4**4+Nj_hvy*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_mcd = Ni_mcd*4**4+Nj_mcd*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_wdy = Ni_wdy*4**4+Nj_wdy*4**3+disc_pop*4**2+disc_val*4**1+disc_inc
stateID_other = np.hstack([stateID_aw,stateID_bk,stateID_hvy,stateID_wdy])

ai_aw = -1*np.int_(a_aw<0) + 0*np.int_(a_aw==0) + 1*np.int_(a_aw>0)+1
ai_bk = -1*np.int_(a_bk<0) + 0*np.int_(a_bk==0) + 1*np.int_(a_bk>0) +1
ai_hvy = -1*np.int_(a_hvy<0) + 0*np.int_(a_hvy==0) + 1*np.int_(a_hvy>0)+1
ai_mcd = -1*np.int_(a_mcd<0) + 0*np.int_(a_mcd==0) + 1*np.int_(a_mcd>0)+1
ai_wdy = -1*np.int_(a_wdy<0) + 0*np.int_(a_wdy==0) + 1*np.int_(a_wdy>0)+1
ai_other = np.hstack([ai_aw,ai_bk,ai_hvy,ai_wdy])

In [ ]:
numstates = 5 # ni,nj,pop,val,inc
states_full = np.arange(4**numstates)
Ni = states_full//(4**(numstates-1))                       #Number of own outlets: {0,1,2,3+}.
Nj = states_full%(4**(numstates-1))//(4**(numstates-2))    #Number of rival outlets: {0,1,2,3+}.
dz1 = states_full%(4**(numstates-2))//(4**(numstates-3))   #Discretized population state: {0,1,2,3}.
dz2 = states_full%(4**(numstates-3))//(4**(numstates-4))   #Discretized property value state: {0,1,2,3}.
dz3 = states_full%(4**(numstates-4))                       #Discretized income state: {0,1,2,3}.
print(np.sum(np.abs(Ni*4**4+Nj*4**3+dz1*4**2+dz2*4**1+dz3-states_full)))
states = np.stack([Ni,Nj,dz1,dz2,dz3],axis=1)

0


In [ ]:
import pandas as pd
filename = '/content/drive/MyDrive/2022_SUMMER/Research_Burger/Picture/'
data_extra = pd.read_csv(filename+'ExtraData.csv')
gdp = data_extra['GDP_Grwoth_Rate']

obser = 2
disc_gdp = np.int_(gdp > 0) #+ np.int_(gdp > gdp75) 
trans_gdp = np.zeros([obser,obser],dtype=int)

for i in range(34):
  trans_gdp[disc_gdp[i],disc_gdp[i+1]] +=1
print(trans_gdp)
trans_gdp = trans_gdp/np.kron(np.ones([1,obser]),np.sum(trans_gdp,axis=1).reshape([-1,1]))
print(trans_gdp)
disc_gdp = (np.repeat(disc_gdp[0:TimePeriod],SamplePath).reshape([TimePeriod,SamplePath]).T).reshape(-1)

[[ 1  2]
 [ 2 29]]
[[0.33333333 0.66666667]
 [0.06451613 0.93548387]]


# Function

##1. Global transtion F_Ni, F_dz1,F_dz2,F_dz3

In [ ]:
I = 2 #MCD and Other homogenous
J = 3 #Action: 0,1,2
NS = 4**(numstates)

In [ ]:
#% F_Ni: Own # of shops changes deterministically, as follows
F_Ni = np.zeros([I,J,obser,NS,NS])#;
for i in range(obser): 
  F_Ni[0,0,i,:,:] = np.array([[1]*4**4+[0]*4**4 + [0] *4**4+[0]*4**4]*4**4 +\
                            [[1]*4**4+[0]*4**4 + [0] *4**4+[0]*4**4]*4**4 + \
                            [[0]*4**4+[1]*4**4 + [0] *4**4+[0]*4**4]*4**4 + \
                            [[0]*4**4+[0]*4**4 + [1] *4**4+[0]*4**4]*4**4 )
  F_Ni[0,1,i,:,:] =np.array([[1]*4**4+[0]*4**4 + [0] *4**4+[0]*4**4]*4**4 +\
                            [[0]*4**4+[1]*4**4 + [0] *4**4+[0]*4**4]*4**4 + \
                            [[0]*4**4+[0]*4**4 + [1] *4**4+[0]*4**4]*4**4 + \
                            [[0]*4**4+[0]*4**4 + [0] *4**4+[1]*4**4]*4**4 )
  F_Ni[0,2,i,:,:] = np.array([[0]*4**4+[1]*4**4 + [0] *4**4+[0]*4**4]*4**4 +\
                            [[0]*4**4+[0]*4**4 + [1] *4**4+[0]*4**4]*4**4 + \
                            [[0]*4**4+[0]*4**4 + [0] *4**4+[1]*4**4]*4**4 + \
                            [[0]*4**4+[0]*4**4 + [0] *4**4+[1]*4**4]*4**4 )
F_Ni[1] = F_Ni[0]
# F_Ni[1:5,0,:,:] = np.stack([F_Ni[0,0,:,:],F_Ni[0,0,:,:],F_Ni[0,0,:,:],F_Ni[0,0,:,:]],axis=0)
# F_Ni[1:5,1,:,:] = np.stack([F_Ni[0,1,:,:],F_Ni[0,1,:,:],F_Ni[0,1,:,:],F_Ni[0,1,:,:]],axis=0)
# F_Ni[1:5,2,:,:] = np.stack([F_Ni[0,2,:,:],F_Ni[0,2,:,:],F_Ni[0,2,:,:],F_Ni[0,2,:,:]],axis=0)

disc_pop_mx = disc_pop[tertile==mk].reshape([-1,TimePeriod])
disc_pop_bk = disc_pop_mx[:,0:TimePeriod-1].reshape(-1)
disc_pop_fw = disc_pop_mx[:,1:TimePeriod].reshape(-1)
disc_val_mx = disc_val[tertile==mk].reshape([-1,TimePeriod])
disc_val_bk = disc_val_mx[:,0:TimePeriod-1].reshape(-1)
disc_val_fw = disc_val_mx[:,1:TimePeriod].reshape(-1)
disc_inc_mx = disc_inc[tertile==mk].reshape([-1,TimePeriod])
disc_inc_bk = disc_inc_mx[:,0:TimePeriod-1].reshape(-1)
disc_inc_fw = disc_inc_mx[:,1:TimePeriod].reshape(-1)

#% F_dz: Make simple & intuitive (4x4) versions for exposition purposes
fz1_4x4 = np.zeros([4,4])#;
fz2_4x4 = np.zeros([4,4])#;
fz3_4x4 = np.zeros([4,4])#;

for x0 in range(4):
    for x1 in range(4):
        numer1 = np.sum((disc_pop_fw == x1) * (disc_pop_bk == x0))#;
        numer2 = np.sum((disc_val_fw == x1) * (disc_val_bk == x0))#;
        numer3 = np.sum((disc_inc_fw == x1) * (disc_inc_bk == x0))#;
        denom1 = np.sum((disc_pop_bk == x0))#;
        denom2 = np.sum((disc_val_bk == x0))#;
        denom3 = np.sum((disc_inc_bk == x0))#;

        fz1_4x4[x0,x1] = numer1 / denom1#;
        fz2_4x4[x0,x1] = numer2 / denom2#;
        fz3_4x4[x0,x1] = numer3 / denom3#;

F_dz1 = np.kron((fz1_4x4[dz1,:])[:,dz1],np.ones([I,J,obser,1,1]))#; % The transition pattern is common across (i,j,mtype)
F_dz2 = np.kron((fz2_4x4[dz2,:])[:,dz2],np.ones([I,J,obser,1,1]))#; % The transition pattern is common across (i,j,mtype)
F_dz3 = np.kron((fz3_4x4[dz3,:])[:,dz3],np.ones([I,J,obser,1,1]))#; % The transition pattern is common across (i,j,mtype)
print(fz1_4x4)
print(fz2_4x4)
print(fz3_4x4)

[[0.95733788 0.04266212 0.         0.        ]
 [0.         0.94048884 0.05951116 0.        ]
 [0.         0.         0.94936709 0.05063291]
 [0.         0.         0.0011554  0.9988446 ]]
[[0.92528736 0.07471264 0.         0.        ]
 [0.         0.91561181 0.08438819 0.        ]
 [0.         0.00250627 0.90225564 0.0952381 ]
 [0.         0.         0.01733577 0.98266423]]
[[0.89330544 0.10669456 0.         0.        ]
 [0.0123348  0.87312775 0.11453744 0.        ]
 [0.         0.01710171 0.88748875 0.09540954]
 [0.         0.         0.01590909 0.98409091]]


## 2. Generate P(nj|policy)

In [ ]:
# % Calculate F from P (& global transition matrices F_Ni, F_dz1, F_dz2) 
def updateF(P):
  F_Nj = np.ones([I,J,obser,NS,NS])#;
  j = 0#;  % F_Nj does not vary with own choice (j = 1:J), so just fill 'j = 1' case & copy
  #% For a given combination of (Ni, dz1, dz2, mtype), make the state indexes (x, x0, x1, x2, x3)
  #% Beliefs: mapping from today's state (Ni,Nj,dz1,dz2) to tomorrow's Nj = {0,1,2,3}
  fnj_mcd = np.zeros([obser,NS,4])#;      % McDonald's belief
  fnj_other = np.zeros([obser,NS,4])#;    % Other 4 chains beliefs
  # for ni in range(4):
  #     for z1 in range(4):
  #         for z2 in range(4):
  for i in range(obser):
    nj = 0#;         % If today's # of rivals = 0
    x =  (4**4)*Ni + (4**3)*nj + (4**2)*dz1 + 4*dz2+dz3#;  % index of today's own state
    x0 = (4**4)*0 + (4**3)*Ni + (4**2)*dz1 + 4*dz2+dz3#;  % index of today's state for rivals with 0 shops

    fnj_mcd[i,x,0] = P[1,1,i,x0]**4#;                               % Prob(Nj'= 0) from McD's perspective
    #print(np.sum(P[1,1,x0]!=0))

    fnj_mcd[i,x,1] = 4 * P[1,2,i,x0] * (P[1,1,i,x0]**3)#;       % Prob(Nj'= 1) from McD's perspective
    fnj_mcd[i,x,2] = 6 * (P[1,2,i,x0]**2) * (P[1,1,i,x0]**2)#;   % Prob(Nj'= 2) from McD's perspective
    fnj_mcd[i,x,3] = 4 * (P[1,2,i,x0]**3) * P[1,1,i,x0]#;       % Prob(Nj'= 3) from McD's perspective
    
    fnj_other[i,x,0] = P[0,1,i,x0] * (P[1,1,i,x0]**3)#;         % Prob(Nj'= 0) etc. from Others' perspectives
    fnj_other[i,x,1] = P[0,2,i,x0] * (P[1,1,i,x0]**3) + P[0,1,i,x0] * 3 * P[1,2,i,x0] * (P[1,1,i,x0]**2)#;  
    fnj_other[i,x,2] = P[0,2,i,x0] * 3 * P[1,2,i,x0] * (P[1,1,i,x0]**2) + P[0,1,i,x0] * 3 * (P[1,2,i,x0]**2) * P[1,1,i,x0]
    fnj_other[i,x,3] = P[0,2,i,x0] * 3 * (P[1,2,i,x0]**2) * P[1,1,i,x0] + P[0,1,i,x0] * (P[1,2,i,x0]**3)
    
    nj = 1#;         % If today's # of rivals = 1                               
    x =  (4**4)*Ni + (4**3)*nj + (4**2)*dz1 +4*dz2+dz3#;  % index of today's own state
    x0 = (4**4)*0 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 0 shops
    x1 =  (4**4)*1 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 1 shops
    
    fnj_mcd[i,x,0] = P[1,0,i,x1] * (P[1,1,i,x0]**3)#;
    fnj_mcd[i,x,1] = P[1,1,i,x1] * (P[1,1,i,x0]**3)#;
    fnj_mcd[i,x,2] = P[1,2,i,x1] * (P[1,1,i,x0]**3) + P[1,1,i,x1] * 3 * P[1,2,i,x0] * (P[1,1,i,x0]**2)#;
    fnj_mcd[i,x,3] = P[1,2,i,x1] * 3 * P[1,2,i,x0] * (P[1,1,i,x0]**2) + P[1,1,i,x1] * 3 * (P[1,2,i,x0]**2) * P[1,1,i,x0]
    
    fnj_other[i,x,0] = .5 * (P[0,0,i,x1] * P[1,1,i,x0]**3) + .5 * (P[0,1,i,x0] * P[1,0,i,x1] * (P[1,1,i,x0]**2))
    fnj_other[i,x,1] = .5 * (P[0,1,i,x1] * P[1,1,i,x0]**3) + .5 * (P[0,1,i,x0] * P[1,1,i,x1] * (P[1,1,i,x0]**2))
    fnj_other[i,x,2] = .5 * (P[0,2,i,x1] * (P[1,1,i,x0]**3) + P[0,1,i,x1] * 3 * P[1,2,i,x0] * (P[1,1,i,x0]**2))+\
                               .5 * (P[0,2,i,x0] * P[1,1,i,x1] * (P[1,1,i,x0]**2) +\
                          P[0,1,i,x0] * (P[1,2,i,x1] * (P[1,1,i,x0]**2) + P[1,1,i,x1] * 2 * P[1,2,i,x0] * P[1,1,i,x0]))
    fnj_other[i,x,3] = .5 * (P[0,2,i,x1] * 3 * P[1,2,i,x0] * (P[1,1,i,x0]**2)\
                          + P[0,1,i,x1] * 3 * (P[1,2,i,x0]**2) * P[1,1,i,x0])\
                          + .5 * (P[0,2,i,x0] * P[1,2,i,x1] * (P[1,1,i,x0]**2)\
                        + P[0,2,i,x0] * P[1,1,i,x1] * 2 * P[1,2,i,x0] * P[1,1,i,x0]\
                        + P[0,1,i,x0] * P[1,2,i,x1] * 2 * P[1,2,i,x0] * P[1,1,i,x0]\
                        + P[0,1,i,x0] * P[1,1,i,x1] * (P[1,2,i,x0]**2))

    nj = 2#;         % If today's # of rivals = 2
    x = (4**4)*Ni + (4**3)*nj + (4**2)*dz1 +4*dz2+dz3#;  % index of today's own state
    x0 = (4**4)*0 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 0 shops
    x1 = (4**4)*1 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 1 shops
    x2 = (4**4)*2 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 2 shops
    
    fnj_mcd[i,x,0] = (P[1,0,i,x1]**2) * (P[1,1,i,x0]**2)
    fnj_mcd[i,x,1] = (1/3) * (P[1,0,i,x2] * (P[1,1,i,x0]**3)) + (2/3) * (P[1,0,i,x1] * P[1,1,i,x1] * (P[1,1,i,x0]**2))
    fnj_mcd[i,x,2] = (1/3) * (P[1,1,i,x2] * (P[1,1,i,x0]**3)) + (2/3) * ((P[1,1,i,x1]**2) * (P[1,1,i,x0]**2))
    fnj_mcd[i,x,3] = (1/3) * (P[1,2,i,x2] * (P[1,1,i,x0]**3)  + (P[1,1,i,x2] * 3 * P[1,2,i,x0] * (P[1,1,i,x0]**2)))
    
    fnj_other[i,x,0] = .5 * 0 + .5 * (P[0,0,i,x1] * P[1,0,i,x1] * (P[1,1,i,x0]**2))
    fnj_other[i,x,1] = .5 * (P[0,0,i,x2] * (P[1,1,i,x0]**3))\
        + .5 * (P[0,0,i,x1] * P[1,1,i,x1] * (P[1,1,i,x0]**2)\
        + P[0,1,i,x1] * P[1,0,i,x1] * (P[1,1,i,x0]**2))
    fnj_other[i,x,2] = .5 * (P[0,1,i,x2] * (P[1,1,i,x0]**3))\
        + .5 * (P[0,1,i,x1] * P[1,1,i,x1] * (P[1,1,i,x0]**2))
    fnj_other[i,x,3] = .5 * (P[0,2,i,x2] * (P[1,1,i,x0]**3)\
        + P[0,1,i,x2] * 3 * P[1,2,i,x0] * P[1,1,i,x0])\
        + .5 * (P[0,2,i,x1] * P[1,1,i,x1] * (P[1,1,i,x0]**2)\
        + P[0,1,i,x1] * P[1,2,i,x1] * (P[1,1,i,x0]**2)\
        + P[0,1,i,x1] * P[1,1,i,x1] * 2 * P[1,2,i,x0] * P[1,1,i,x0])
    
    nj = 3#;         % If today's # of rivals = 3
    x = (4**4)*Ni + (4**3)*nj + (4**2)*dz1 +4*dz2+dz3#;  % index of today's own state
    x0 = (4**4)*0 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 0 shops
    x1 = (4**4)*1 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 1 shops
    x2 = (4**4)*2 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 2 shops
    x3 = (4**4)*3 + (4**3)*Ni + (4**2)*dz1 +4*dz2+dz3#;  % index of today's state for rivals with 3 shops
    
    fnj_mcd[i,x,0] = (2/3) * (P[1,0,i,x1]**3) * P[1,1,i,x0]
    fnj_mcd[i,x,1] = (2/3) * (P[1,0,i,x1]**2) * P[1,1,i,x1] + (1/3) * P[1,0,i,x2] * P[1,0,i,x1] * (P[1,1,i,x0]**2)
    fnj_mcd[i,x,2] = (2/3) * P[1,0,i,x1] * (P[1,1,i,x1]**2) + (1/3) * (P[1,0,i,x2] * P[1,1,i,x1] * (P[1,1,i,x0]**2)\
        + P[1,1,i,x2] * P[1,0,i,x1] * (P[1,1,i,x0]**2))
    fnj_mcd[i,x,3] = (2/3) * (P[1,1,i,x1]**3) * P[1,1,i,x0] + (1/3) * P[1,1,i,x2] * P[1,1,i,x1] * (P[1,1,i,x0]**2)
    
    fnj_other[i,x,0] = .25 * (P[0,0,i,x1] * (P[1,0,i,x1]**2) * P[1,1,i,x0])
    fnj_other[i,x,1] = .25 * 0 + .25 * (P[0,0,i,x2] * P[1,0,i,x1] * (P[1,1,i,x0]**2))\
        + .25 * (P[0,0,i,x1] * 2 * P[1,0,i,x1] * P[1,1,i,x1] * P[1,1,i,x0]\
        + P[0,1,i,x1] * (P[1,0,i,x1]**2) * P[1,1,i,x0])\
        + .25 * (P[0,0,i,x1] * P[1,0,i,x2] * (P[1,1,i,x0]**2))
    fnj_other[i,x,2] = .25 * (P[0,0,i,x3] * (P[1,1,i,x0]**3))\
        + .25 * (P[0,0,i,x2] * P[1,1,i,x1] * (P[1,1,i,x0]**2)\
        + P[0,1,i,x2] * P[1,0,i,x1] * (P[1,1,i,x0]**2))\
        + .25 * (P[0,0,i,x1] * (P[1,1,i,x1]**2) * P[1,1,i,x0]\
        + P[0,1,i,x1] * 2 * P[1,0,i,x1] * P[1,1,i,x1] * P[1,1,i,x0])\
        + .25 * (P[0,0,i,x1] * P[1,1,i,x2] * (P[1,1,i,x0]**2)\
        + P[0,1,i,x1] * P[1,0,i,x2] * (P[1,1,i,x0]**2))
    fnj_other[i,x,3] = .25 * (P[0,1,i,x3] * (P[1,1,i,x0]**3))\
        + .25 * (P[0,1,i,x2] * P[1,1,i,x1] * (P[1,1,i,x0]**2))\
        + .25 * (P[0,1,i,x1] * (P[1,1,i,x1]**2) * P[1,1,i,x0])\
        + .25 * (P[0,1,i,x1] * P[1,1,i,x2] * (P[1,1,i,x0]**2))
              
  Denom_mcd = np.kron(np.reshape(np.sum(fnj_mcd,axis=2),[obser,-1,1]),np.ones([1,1,4]))#;         % Sum of each row
  fnj_mcd = fnj_mcd / Denom_mcd#;                 % Make sure each row sums up to 1
  fnj_mcd[np.isnan(fnj_mcd)] = 0#;                    % Replace NaN with 0
  F_Nj[0,j] = fnj_mcd[:,:,Nj]

  Denom_other = np.kron(np.reshape(np.sum(fnj_other,axis=2),[obser,-1,1]),np.ones([1,1,4]))#;             % Sum of each row
  fnj_other = fnj_other / Denom_other#;                   % Make sure each row sums up to 1
  fnj_other[np.isnan(fnj_other)] = 0#;                        % Replace NaN with 0
  F_Nj[1,j] = fnj_other[:,:,Nj]

  #F_Nj[2:5,j,:,:] = np.stack([F_Nj[1,j,:,:] for copy in range(3)],axis=0)#;    % Other 4 chains are symmetric

  for j in range(1,J):    
      F_Nj[:,j] = F_Nj[:,0]    
  F = F_Ni * F_Nj * F_dz1 * F_dz2 *F_dz3
  return F
from scipy.special import softmax
P_new = 1/J*np.ones([I,J,obser,NS])#softmax(np.arange(I*J*NS).reshape([I,J,NS]),axis=1)
F_new = updateF(P_new.copy())
#np.sum(F_new!=0)

##3.Value Iteration

In [ ]:
from scipy.special import logsumexp
from scipy.special import softmax

def updatePi(Q):
  P = np.zeros([I,J,obser,NS])#; player, action, states, Market Type
  P[0] = softmax(Q[0],axis=0)
  P[1] = softmax(Q[1],axis=0)
  P[np.isnan(P)] = 0#;
  return P
def updateQ(thetaS,Q,F,beta = 0.9):
  Q_new = np.zeros([I,J,obser,NS])
  gamma = 0.5772
  
  thetaS_mcd = np.reshape(thetaS[0:int(len(thetaS)/2)-2],-1)
  thetaS_other = np.reshape(thetaS[int(len(thetaS)/2):len(thetaS)-2],-1)
  for i in range(obser):
    Q_new[0,2,i] =  np.stack([Ni,Ni *Ni,Ni*Nj,Ni*dz1,Ni*dz2,Ni*dz3, -1* np.ones(NS)],axis=1).dot(thetaS_mcd) +\
                    Ni*((i==0)*thetaS[int(len(thetaS)/2)-2] + (i==1)*thetaS[int(len(thetaS)/2)-1])
    Q_new[0,1,i] =np.stack([Ni,Ni *Ni,Ni*Nj,Ni*dz1,Ni*dz2,Ni*dz3, 0* np.ones(NS)],axis=1).dot(thetaS_mcd)  +\
                    Ni*((i==0)*thetaS[int(len(thetaS)/2)-2] + (i==1)*thetaS[int(len(thetaS)/2)-1])
    Q_new[0,0,i] = Q_new[0,1,i].copy()

    Q_new[1,2,i] =  np.stack([Ni,Ni *Ni,Ni*Nj,Ni*dz1,Ni*dz2,Ni*dz3, -1* np.ones(NS)],axis=1).dot(thetaS_other) +\
                  Ni*((i==0)*thetaS[len(thetaS)-2] + (i==1)*thetaS[len(thetaS)-1])
    Q_new[1,1,i] =np.stack([Ni,Ni *Ni,Ni*Nj,Ni*dz1,Ni*dz2,Ni*dz3, 0* np.ones(NS)],axis=1).dot(thetaS_other)+\
                  Ni*((i==0)*thetaS[len(thetaS)-2] + (i==1)*thetaS[len(thetaS)-1])
    Q_new[1,0,i] = Q_new[1,1,i].copy()
  
  v_mcd = gamma + logsumexp(Q[0],axis=0) #gdp_new,NS_new
  v_other = gamma + logsumexp(Q[1],axis=0) #gdp_new,NS_new
  #v_mcd = v_mcd.reshape([-1,1])
  #v_other =v_other.reshape([-1,1])
  for j in range(J):
    #print(np.shape(F[0,j].dot(v_mcd)),Q_new[0,j].shape)
    v_temp_mcd = F[0,j].dot(v_mcd.T)# gdp_old, ns_old, (ns_new \times ns_new), gdp_new
    v_temp_other = F[1,j].dot(v_other.T)
    for i in range(obser):
      Q_new[0,j,i] = Q_new[0,j,i] + beta*(v_temp_mcd[i,:,0]*trans_gdp[i,0]+v_temp_mcd[i,:,1]*trans_gdp[i,1]) 
      Q_new[1,j,i] = Q_new[1,j,i] + beta*(v_temp_other[i,:,0]*trans_gdp[i,0]+v_temp_other[i,:,1]*trans_gdp[i,1])
  return Q_new


def ValueIteration(thetaS,error = 5e-2,beta = 0.6):
  Pi0 = 1/3*np.ones([I,J,obser,NS])
  F0= updateF(Pi0)
  Q0 = np.zeros([I,J,obser,NS])
  Pi_old = Pi0
  F_old = F0
  Q_old = Q0
  for iter in range(1000):
    Q_new = updateQ(thetaS,Q_old,F_old,beta=beta) #I, J, NS
    Pi_new = updatePi(Q_new)#I,J,NS
    F_new = updateF(Pi_new) # I, J, NS,NS

    error1 = np.sum(np.abs(Q_new-Q_old))/np.max(np.abs(Q_old))
    error2 = np.sum(np.abs(Pi_new-Pi_old))/np.max(np.abs(Pi_old))
    error3 = np.sum(np.abs(F_new-F_old))/np.max(np.abs(F_old))
    #print('iteration {} --> e1:{},e2:{},e3:{}'.format(iter,error1,error2,error3))
    if error1<error and error2<error and error3<error:
      #print('iteration {} --> e1:{},e2:{},e3:{}'.format(iter,error1,error2,error3))
      break
    Q_old = Q_new.copy()
    Pi_old = Pi_new.copy()
    F_old = F_new.copy()
  return Q_new, Pi_new,F_new
# thetaS = np.array([2.09932422e+01,  9.39319015e-02, -2.55199026e-01,  5.22040762e-02, 3.74165364e-01,  1.85959336e+02,
# 2.07684086, -1.34568641, -0.06689713, -0.09297975, -0.07754167,9.49715754])
# thetaS = np.zeros((7+2)*2)
# Q_new, Pi_new,F_new = ValueIteration(thetaS)

##4. log-likelihood

In [ ]:
disc_gdp_other = np.hstack([disc_gdp for i in range(4)])
disc_gdp_mcd = disc_gdp.copy()

In [ ]:
tertile_other = np.hstack([tertile,tertile,tertile,tertile])
def func(thetaS):
  thetaS = np.hstack([thetaS[0:16],0,thetaS[16]])
  _, Pi_new,_ = ValueIteration(thetaS)
  #print(np.unique(a_mcd))
  res = -np.sum(np.log(Pi_new[0,ai_mcd[tertile==mk],disc_gdp_mcd[tertile==mk],stateID_mcd[tertile==mk]]))
  res = res -np.sum(np.log(Pi_new[1,ai_other[tertile_other==mk],disc_gdp_other[tertile_other==mk],stateID_other[tertile_other==mk]]))
  print(thetaS,res)
  return res/5/MT
from scipy.optimize import minimize
thetaS = np.zeros((7+2)*2-1)
print('start from:',thetaS )
res22 = minimize(func,thetaS)#,tol=1e-3)
print(res22,res22.fun*5*MT) # 28min

start from: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: RuntimeWarning: divide by zero encountered in double_scalars


Streaming output truncated to the last 5000 lines.
  0.00000000e+00  1.59721502e+00] 16087.027821420867
[-1.02117134e-02 -1.29725506e+00 -1.27392098e+00  5.69946355e-01
  8.30129411e-01  5.58378582e-01  6.80961606e+00 -1.47250461e-01
  1.37041734e-01  1.68710528e+00 -1.15800969e+00 -1.00548956e+00
  2.57787439e-01  1.77609539e-01  9.50625543e-04  7.76289887e+00
  0.00000000e+00  1.59721502e+00] 16087.027821103402
[-1.02117134e-02 -1.29725506e+00 -1.27392098e+00  5.69946355e-01
  8.30129411e-01  5.58378582e-01  6.80961604e+00 -1.47250446e-01
  1.37041734e-01  1.68710528e+00 -1.15800969e+00 -1.00548956e+00
  2.57787439e-01  1.77609539e-01  9.50625543e-04  7.76289887e+00
  0.00000000e+00  1.59721502e+00] 16087.027821212905
[-1.02117134e-02 -1.29725506e+00 -1.27392098e+00  5.69946355e-01
  8.30129411e-01  5.58378582e-01  6.80961604e+00 -1.47250461e-01
  1.37041748e-01  1.68710528e+00 -1.15800969e+00 -1.00548956e+00
  2.57787439e-01  1.77609539e-01  9.50625543e-04  7.76289887e+00
  0.000000

In [1]:
import numpy as np
res1 = np.array([ 6.42124416, -8.31112455, -1.55239131,  0.40433173,  0.58599092,
        0.29775255, 12.69616967, -0.51173718,  6.93295814,  1.59927561,
       -1.08650371, -0.55817754,  0.21288478,  0.1272763 ,  0.03320988,
        7.66361182,  1.53894682]) 
res2 =16057.570748563197
print(['{:2.6f}'.format(res1[i]) for i in range(len(res1))])
print('{:6.6f}'.format(-res2))

['6.421244', '-8.311125', '-1.552391', '0.404332', '0.585991', '0.297753', '12.696170', '-0.511737', '6.932958', '1.599276', '-1.086504', '-0.558178', '0.212885', '0.127276', '0.033210', '7.663612', '1.538947']
-16057.570749
